In [1]:
import gensim

model_sg = gensim.models.KeyedVectors.load_word2vec_format("skipgram.vec", binary=False)
model_sg.init_sims(replace=True)


model_cbow = gensim.models.KeyedVectors.load_word2vec_format("cbow.txt", binary=False)
model_cbow.init_sims(replace=True)

/var/folders/r_/mggr546j3qjgz1bx6blb249h0000gq/T/ipykernel_51739/1912045750.py:4: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_sg.init_sims(replace=True)
/var/folders/r_/mggr546j3qjgz1bx6blb249h0000gq/T/ipykernel_51739/1912045750.py:8: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_cbow.init_sims(replace=True)


In [2]:
import pandas as pd
import pymorphy3

analyzer = pymorphy3.analyzer.MorphAnalyzer()
wordsim_dataset = pd.read_csv('hj-wordsim353-similarity.csv')



wordsim_dataset['word1_attr'] =  wordsim_dataset['word1'].apply(func=lambda w: f'{analyzer.parse(w)[0].normal_form.replace("ё", "е")}_{analyzer.parse(w)[0].tag.POS}')
wordsim_dataset['word2_attr'] =  wordsim_dataset['word2'].apply(func=lambda w: f'{analyzer.parse(w)[0].normal_form.replace("ё", "е")}_{analyzer.parse(w)[0].tag.POS}')

In [3]:
wordsim_dataset.head(10)

,word1,word2,sim,word1_attr,word2_attr
0,маг,волшебник,0.958333,маг_NOUN,волшебник_NOUN
1,машина,автомобиль,0.952381,машина_NOUN,автомобиль_NOUN
2,мальчик,парень,0.952381,мальчик_NOUN,парень_NOUN
3,доллар,бакс,0.952381,доллар_NOUN,бакс_NOUN
4,расчет,вычисление,0.916667,расчет_NOUN,вычисление_NOUN
5,побережье,берег,0.904762,побережье_NOUN,берег_NOUN
6,жидкость,вода,0.904762,жидкость_NOUN,вода_NOUN
7,чемпионат,турнир,0.888889,чемпионат_NOUN,турнир_NOUN
8,тигр,тигр,0.875000,тигр_NOUN,тигр_NOUN
9,поездка,путешествие,0.857143,поездка_NOUN,путешествие_NOUN


In [10]:
import numpy as np

result_cbow = pd.DataFrame(columns=['word1', 'word2', 'model_similarity', 'dataset_similarity'])

for _, row in wordsim_dataset.iterrows():
    w1 = row.iloc[3].replace("ADJF", "ADJ").replace("NUMR", "NUM").replace("INFN", 'VERB')
    w2 = row.iloc[4].replace("ADJF", "ADJ").replace("NUMR", "NUM").replace("INFN", 'VERB')
    if w1 not in model_cbow.key_to_index:
        result_cbow.loc[len(result_cbow)] = [w1, w2, 0, expected_similarity]
        continue
    if w2 not in model_cbow:
        result_cbow.loc[len(result_cbow)] = [w1, w2, 0, expected_similarity]
        continue
    cbow_similarity = model_cbow.similarity(w1, w2)
    expected_similarity = row.iloc[2]

    result_cbow.loc[len(result_cbow)] = [w1, w2, cbow_similarity, expected_similarity]

In [11]:
result_cbow.head(100)

,word1,word2,model_similarity,dataset_similarity
0,маг_NOUN,волшебник_NOUN,0.591843,0.958333
1,машина_NOUN,автомобиль_NOUN,0.787189,0.952381
2,мальчик_NOUN,парень_NOUN,0.466987,0.952381
3,доллар_NOUN,бакс_NOUN,0.724421,0.952381
4,расчет_NOUN,вычисление_NOUN,0.584030,0.916667
...,...,...,...,...
95,монах_NOUN,оракул_NOUN,0.242732,0.307692
96,начало_NOUN,год_NOUN,0.328140,0.303030
97,побережье_NOUN,лес_NOUN,0.338544,0.300000
98,умник_NOUN,студент_NOUN,0.125002,0.291667


In [13]:
result_sg = pd.DataFrame(columns=['word1', 'word2', 'model_similarity', 'dataset_similarity'])

for _, row in wordsim_dataset.iterrows():
    w1 = row.iloc[3].replace("ADJF", "ADJ").replace("NUMR", "NUM").replace("INFN", 'VERB')
    w2 = row.iloc[4].replace("ADJF", "ADJ").replace("NUMR", "NUM").replace("INFN", 'VERB')
    if w1 not in model_sg.key_to_index:
        result_sg.loc[len(result_sg)] = [w1, w2, 0, expected_similarity]
        continue
    if w2 not in model_sg.key_to_index:
        result_sg.loc[len(result_sg)] = [w1, w2, 0, expected_similarity]
        continue
    sg_similarity = model_sg.similarity(w1, w2)
    expected_similarity = row.iloc[2]

    result_sg.loc[len(result_sg)] = [w1, w2, sg_similarity, expected_similarity]

In [14]:
result_sg.head(100)

,word1,word2,model_similarity,dataset_similarity
0,маг_NOUN,волшебник_NOUN,0.560064,0.958333
1,машина_NOUN,автомобиль_NOUN,0.789657,0.952381
2,мальчик_NOUN,парень_NOUN,0.594909,0.952381
3,доллар_NOUN,бакс_NOUN,0.705023,0.952381
4,расчет_NOUN,вычисление_NOUN,0.577726,0.916667
...,...,...,...,...
95,монах_NOUN,оракул_NOUN,0.279536,0.307692
96,начало_NOUN,год_NOUN,0.383139,0.303030
97,побережье_NOUN,лес_NOUN,0.393032,0.300000
98,умник_NOUN,студент_NOUN,0.199925,0.291667


In [15]:
import scipy.stats


correlation1, p_value1 = scipy.stats.spearmanr(wordsim_dataset['sim'].to_list(), result_cbow['model_similarity'].to_list())
print(f"Корреляция для Модели 1 (cbow): {correlation1:.3f} (p-value: {p_value1:.3f})")


correlation2, p_value2 = scipy.stats.spearmanr(wordsim_dataset['sim'].to_list(), result_sg['model_similarity'].to_list())
print(f"Корреляция для Модели 2 (skipgram): {correlation2:.3f} (p-value: {p_value2:.3f})")

Корреляция для Модели 1 (cbow): 0.672 (p-value: 0.000)
Корреляция для Модели 2 (skipgram): 0.706 (p-value: 0.000)
